In [1]:
#dependencies
import pandas as pd
import os
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
import sqlite3
import csv

## Import into Pandas

In [2]:
# Load 2017 ACS Population Estimates
file_to_load = 'Resources/nst-est2018-alldata.csv'
pop_data = pd.read_csv(file_to_load)
pop_data.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,...,RDOMESTICMIG2017,RDOMESTICMIG2018,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018
0,10,0,0,0,United States,308745538,308758105,309326085,311580009,313874218,...,0.000000,0.000000,2.553948,2.746049,2.701727,2.988276,3.328598,3.321549,2.941086,3.001086
1,20,1,0,0,Northeast Region,55317240,55318430,55380645,55600532,55776729,...,-5.651919,-5.222289,0.845134,0.040762,-0.397011,-0.923951,-2.011735,-2.430459,-1.801582,-1.127222
2,20,2,0,0,Midwest Region,66927001,66929743,66974749,67152631,67336937,...,-2.370672,-2.301663,-1.043009,-0.896575,0.042505,-0.715656,-1.357662,-1.226811,-0.519621,-0.431833
3,20,3,0,0,South Region,114555744,114563045,114867066,116039399,117271075,...,2.963135,2.779373,5.379667,5.836112,5.290067,6.206402,7.328494,7.225046,6.252425,6.148925
4,20,4,0,0,West Region,71945553,71946887,72103625,72787447,73489477,...,1.478565,1.350094,2.689358,3.226360,3.343874,4.148127,5.127995,5.372314,4.164981,3.965769


In [3]:
#import Gun Counts txt file and transform into a csv format
gc_txt_path = os.path.join("Resources","raw-thoughtco-guns-by-state.txt")
gc_path = os.path.join("Resources","raw-thoughtco-guns-by-state_transformed.txt")
with open(gc_txt_path) as fin, open(gc_path,'w') as fout:
    for line in fin:
        fout.write(line.replace(',','').replace('\t',','))

#open new file as csv
gc_df= pd.read_csv(gc_path)
print(gc_df.columns,len(gc_df))
gc_df.head()

Index(['Rank', 'State', '# of guns per capita', '# of guns registered'], dtype='object') 51


,Rank,State,# of guns per capita,# of guns registered
0,1,Wyoming,229.24,132806
1,2,Washington D.C.,68.05,47228
2,3,New Hampshire,46.76,64135
3,4,New Mexico,46.73,97580
4,5,Virginia,36.34,307822


In [16]:
#import Gun Violence CSV into Pandas

# gv_path = os.path.join("Resources","gun-violence-data_01-2013_03-2018.csv")
gv_path_1 = os.path.join("Resources","gun-violence-data_01-2013_03-2018_1.csv")
gv_path_2 = os.path.join("Resources","gun-violence-data_01-2013_03-2018_2.csv")
try:
    print("Try to split files for easy upload")
    gv_df = pd.read_csv(gv_path)
    gv_df_1 = gv_df.iloc[0:round((len(gv_df)/2))]
    gv_df_2 = gv_df.iloc[round((len(gv_df)/2+.5)):len(gv_df)]
    gv_df_1.to_csv(gv_path_1)
    gv_df_2.to_csv(gv_path_2)

except:
    print("Files are already split")
    gv_df_1 = pd.read_csv(gv_path_1)
    gv_df_2 = pd.read_csv(gv_path_2)
    gv_df = pd.concat([gv_df_1,gv_df_2])


print(gv_df.columns)
print(len(gv_df))
gv_df.head()

Try to split files for easy upload
Files are already split
Index(['Unnamed: 0', 'Unnamed: 0.1', 'incident_id', 'date', 'state',
       'city_or_county', 'address', 'n_killed', 'n_injured', 'incident_url',
       'source_url', 'incident_url_fields_missing', 'congressional_district',
       'gun_stolen', 'gun_type', 'incident_characteristics', 'latitude',
       'location_description', 'longitude', 'n_guns_involved', 'notes',
       'participant_age', 'participant_age_group', 'participant_gender',
       'participant_name', 'participant_relationship', 'participant_status',
       'participant_type', 'sources', 'state_house_district',
       'state_senate_district'],
      dtype='object')
239676


,Unnamed: 0,Unnamed: 0.1,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,1,1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,2,2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,3,3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,4,4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [6]:
gv_df_1 = gv_df.iloc[0:round((len(gv_df)/2))]
gv_df_2 = gv_df.iloc[round((len(gv_df)/2+.5)):len(gv_df)]
gv_df_1.to_csv(gv_path_1)
gv_df_2.to_csv(gv_path_2)

In [7]:
# import Educational Attainment CSV into Pandas
edu_path = os.path.join("Resources","wiki_us_edu_attainment.csv")
edu_df = pd.read_csv(edu_path)
print(edu_df.columns)
edu_df.head()

Index(['State', 'Pct High school graduate', 'Pct Bachelor degree',
       'Pct Advanced degree'],
      dtype='object')


,State,Pct High school graduate,Pct Bachelor degree,Pct Advanced degree
0,North Carolina,0.8580,0.2842,0.09900
1,Florida,0.8687,0.2730,0.09796
2,Ohio,0.8910,0.2610,0.09705
3,Nebraska,0.9067,0.2930,0.09705
4,Montana,0.9280,0.2950,0.09500


## Narrow and Transform Data

In [10]:
# Clean 2017 ACS Population Estimates by State 
pop_clean = pop_data[['NAME','POPESTIMATE2017']]
pop_2017 = pop_clean.drop(axis=0, labels=range(5))
pop_2017.columns = ["state","pop_est"]
pop_2017.reset_index(drop=True, inplace=True)
pop_2017.index.name = "id"
pop_2017.head()

,state,pop_est
id,,
0,Alabama,4875120
1,Alaska,739786
2,Arizona,7048876
3,Arkansas,3002997
4,California,39399349


In [11]:
#drop duplicates
gc_df.drop_duplicates()
gv_df.drop_duplicates()
edu_df.drop_duplicates()

#Filter through data frame for columns wanted and dates wanted
gv_df = gv_df[gv_df['date'] > "2016-12-31" ]


gv_cols = ["state", "n_killed", "n_injured", "gun_type"]
new_gv_df = gv_df[gv_cols].copy()
new_gv_df.columns = ["state","deaths", "injured", "gun_type"]
new_gv_df.head()

transformed_gv_df = pd.DataFrame(new_gv_df.groupby("state").sum()).reset_index()
transformed_gv_df.index.name = "id"
transformed_gv_df.head()

# grouped_dv_df


,state,deaths,injured
id,,,
0,Alabama,679,1075
1,Alaska,80,89
2,Arizona,374,371
3,Arkansas,259,520
4,California,1727,2385


In [12]:
gc_df.head()

,Rank,State,# of guns per capita,# of guns registered
0,1,Wyoming,229.24,132806
1,2,Washington D.C.,68.05,47228
2,3,New Hampshire,46.76,64135
3,4,New Mexico,46.73,97580
4,5,Virginia,36.34,307822


In [13]:
edu_df.index.name = "id"
edu_df.head()

,State,Pct High school graduate,Pct Bachelor degree,Pct Advanced degree
id,,,,
0,North Carolina,0.8580,0.2842,0.09900
1,Florida,0.8687,0.2730,0.09796
2,Ohio,0.8910,0.2610,0.09705
3,Nebraska,0.9067,0.2930,0.09705
4,Montana,0.9280,0.2950,0.09500


## Create Database connection and load

In [14]:
conn = "gun_stats_db"
engine = create_engine(f'sqlite:///{conn}')

gc_df.to_sql("gun_counts_state_estimate",engine, if_exists = "replace")
edu_df.to_sql("education_%_state", engine, if_exists = "replace")
transformed_gv_df.to_sql("gun_violence_state",engine,if_exists= "replace")
pop_2017.to_sql("population_est_state",engine,if_exists= "replace")


## Verifiy Upload

In [15]:
sql3_conn = sqlite3.connect('gun_stats_db')

df = pd.read_sql_query("SELECT * FROM population_est_state", sql3_conn)
df.head()

,id,state,pop_est
0,0,Alabama,4875120
1,1,Alaska,739786
2,2,Arizona,7048876
3,3,Arkansas,3002997
4,4,California,39399349
